<a href="https://colab.research.google.com/github/PRAGHATHAPRIYA2589/step/blob/Day-4-Task/Day4_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: generate the code that should use bert for embedding for the sentiment analysis and by using cosine similarity, i need know the  similarity for that

!pip install transformers sentence-transformers

from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer, util
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'  # You can choose other BERT models
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Or use a sentence transformer model directly for better performance
# model = SentenceTransformer('all-mpnet-base-v2') # Example: all-mpnet-base-v2


def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pooling of token embeddings
    return embeddings


def calculate_similarity(text1, text2):
    embedding1 = get_bert_embedding(text1)
    embedding2 = get_bert_embedding(text2)

    # Calculate cosine similarity
    similarity_score = util.cos_sim(embedding1, embedding2).item()
    return similarity_score


# Example usage
text1 = "This is a positive sentence."
text2 = "This is another positive sentence."
text3 = "This is a negative sentence."

similarity_1_2 = calculate_similarity(text1, text2)
similarity_1_3 = calculate_similarity(text1, text3)


print(f"Similarity between '{text1}' and '{text2}': {similarity_1_2}")
print(f"Similarity between '{text1}' and '{text3}': {similarity_1_3}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Similarity between 'This is a positive sentence.' and 'This is another positive sentence.': 0.9624009728431702
Similarity between 'This is a positive sentence.' and 'This is a negative sentence.': 0.9670071005821228


In [ ]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize

# Download the 'punkt_tab' resource
nltk.download('punkt_tab')

# Sample corpus to train Word2Vec
sentences = [
    "This is a positive sentence",
    "This is another positive sentence",
    "This is a negative sentence",
    "I love this product",
    "The movie was terrible"
]

# Tokenize the sentences
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Train Word2Vec model
word2vec_model = Word2Vec(tokenized_sentences, vector_size=50, window=5, min_count=1, workers=4)

# Function to get sentence embeddings by averaging word embeddings
def get_sentence_embedding(sentence, model):
    words = word_tokenize(sentence.lower())
    word_embeddings = [model.wv[word] for word in words if word in model.wv]
    if len(word_embeddings) > 0:
        return np.mean(word_embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)

# Function to calculate cosine similarity
def calculate_similarity(text1, text2):
    embedding1 = get_sentence_embedding(text1, word2vec_model)
    embedding2 = get_sentence_embedding(text2, word2vec_model)
    return cosine_similarity([embedding1], [embedding2])[0][0]

# Example sentences
text1 = "This is a positive sentence"
text2 = "This is another positive sentence"
text3 = "This is a negative sentence"

# Calculate similarities
similarity_1_2 = calculate_similarity(text1, text2)
similarity_1_3 = calculate_similarity(text1, text3)

# Print results
print(f"Similarity between '{text1}' and '{text2}': {similarity_1_2:.4f}")
print(f"Similarity between '{text1}' and '{text3}': {similarity_1_3:.4f}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Similarity between 'This is a positive sentence' and 'This is another positive sentence': 0.8277
Similarity between 'This is a positive sentence' and 'This is a negative sentence': 0.8526


In [2]:
!pip install datasets
# Import the datasets and transformers packages
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np
import pandas as pd

# Load the sms_spam dataset
# The sms_spam dataset only has a 'train' split.
# We load the 'train' split and then manually split it into train and test.
ds = load_dataset("sms_spam", split="train")

# Split the dataset into train and test
train_test_split = ds.train_test_split(test_size=0.2)  # Adjust test_size as needed
ds = {
    "train": train_test_split["train"],
    "test": train_test_split["test"],
}


# Load a pre-trained BERT model and add a classification head
model_name = "bert-base-uncased"  # Specify the desired pre-trained model name
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # Binary classification (spam or not spam)
    id2label={0: "HAM", 1: "SPAM"},
    label2id={"HAM": 0, "SPAM": 1},
)

# Freeze the base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False

# Define the compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Before initializing the data collator, you need to initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name) # using the same model name

# Initialize a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./data/sms_spam_classification",
    learning_rate=3e-5,  # Adjust learning rate for experimentation
    per_device_train_batch_size=8,  # Adjust batch size for experimentation
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Train for 3 epochs
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
)


# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sms'], padding="max_length", truncation=True) # changed 'text' to 'sms'

# Applying .map() to the train and test datasets within the dictionary
tokenized_ds = {
    "train": ds["train"].map(tokenize_function, batched=True),
    "test": ds["test"].map(tokenize_function, batched=True),
}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)

# Add predictions to the dataset for analysis
predictions = trainer.predict(tokenized_ds["test"])

df = pd.DataFrame(ds["test"])
df["predicted_label"] = np.argmax(predictions[0], axis=1)
df["predicted_label"] = df["predicted_label"].map({0: "HAM", 1: "SPAM"})
df["label"] = df["label"].map({0: "HAM", 1: "SPAM"})

# Display mismatched predictions
mismatched = df[df["label"] != df["predicted_label"]]
pd.set_option("display.max_colwidth", None)
print("Mismatched Predictions:")
print(mismatched.head(5))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

<ipython-input-2-0a5b2b08daa4>:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.365100,0.355440,0.866368
2,0.270500,0.335724,0.866368
3,0.391600,0.328807,0.866368


Evaluation Results: {'eval_loss': 0.32880741357803345, 'eval_accuracy': 0.8663677130044843, 'eval_runtime': 34.0014, 'eval_samples_per_second': 32.793, 'eval_steps_per_second': 4.117, 'epoch': 3.0}
Mismatched Predictions:
                                                                                                                                                                     sms  \
0            sports fans - get the latest sports news str* 2 ur mobile 1 wk FREE PLUS a FREE TONE Txt SPORT ON to 8007 www.getzed.co.uk 0870141701216+ norm 4txt/120p \n   
13                                                            Ur HMV Quiz cash-balance is currently £500 - to maximize ur cash-in now send HMV1 to 86688 only 150p/msg\n   
16                                  we tried to contact you re your response to our offer of a new nokia fone and camcorder hit reply or call 08000930705 for delivery\n   
21  Loan for any purpose £500 - £75,000. Homeowners + Tenants welcome. Have you been previ